<a href="https://colab.research.google.com/github/amitkp57/Jupyter/blob/dev/Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Move to GPU mode if you are in Google Colab
Go to `Runtime` -> `Change runtime type` to activate GPU.

### Provide your credentials to the runtime


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Mount Google Drive

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
WORKING_DIRECTORY = '/content/gdrive/MyDrive/Data/Jupyter'
os.environ['WORKING_DIRECTORY'] = WORKING_DIRECTORY
%cd $WORKING_DIRECTORY
!ls -latr

### Clone git repo

In [ ]:
%cd $WORKING_DIRECTORY
# !git clone https://github.com/amitkp57/Jupyter
!git pull
!pip install -r requirements.txt

### Setup meta data

Creates datasets.txt, tables.txt and columns.json in the /data folder. These files are used for querying Google Bigquery tables. 

In [4]:
from scripts.MetaData import save_locally
DATA_PATH = f'{WORKING_DIRECTORY}/data'
save_locally(DATA_PATH)
print('Completed!')